In [1]:
import numpy as np, pandas as pd
import train as t
from src import helper_models as hm, helper_data as hd, helper_tts as htts
from importlib import reload
import keras, glob




Using TensorFlow backend.


In [2]:
# Reload imports of custom modules, in case I am modifying and reloading them live
reload(t)
reload(hd)
reload(hm)
reload(htts)


<module 'src.helper_tts' from '/mnt/volume2Tb/Dropbox/PROJECTS/MACHINE-LEARNING/AQUASCOPE/plankifier/src/helper_tts.py'>

# Initialize simulation

In [2]:
sim=t.Ctrain()

# Init parameters
Parameters must always be updated through the `UpdateParams()` method

In [3]:
def SetModelParams(kind):
    ''' 
    A quick way to set compatible user parameters of data and model kinds
    
    Input: kind = either 'image', 'feat' or 'mixed'
    Output: model_image, model_feat, datakind, ttkind, aug
    '''
    
    if kind == 'image':
        return ('conv2', None, 'image', 'image', True)
    
    elif kind == 'feat':
        return (None, 'mlp', 'feat', 'feat', False)
    
    elif kind == 'mixed':
        return ('conv2', 'mlp', 'mixed', 'mixed', False)
        

In [10]:
model_image, model_feat, datakind, ttkind, aug = SetModelParams('image')

sim.UpdateParams(
    datapaths=['./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/',
               './data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.07.06/'],
    outpath     = 'out_example', 
    L           = 128, 
    aug         = aug, 
    model_feat  = model_feat,
    model_image = model_image,
    datakind    = datakind, 
    ttkind      = ttkind,
    class_select= None #['chaoborus','bosmina','unknown_plankton'] 
    )


Create output directory only after you've set the right `outpath`

In [11]:
sim.CreateOutDir()


In [12]:
print(sim.params.datapaths)


['./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/', './data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.07.06/']


# Load data

In [13]:
# These arguments are the defaults
reload(hd)
sim.LoadData(L=sim.params.L, class_select=sim.params.class_select, datakind=sim.params.datakind) 


datapaths: ['./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/', './data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.07.06/']
classes from datapaths: ['kellikottia', 'copepod_skins', 'uroglena', 'conochilus', 'bosmina', 'nauplius', 'unknown', 'diatom_chain', 'dinobryon', 'ceratium', 'daphnia', 'fragilaria', 'maybe_cyano', 'leptodora', 'daphnia_skins', 'synchaeta', 'fish', 'rotifers', 'trichocerca', 'cyclops', 'polyarthra', 'chaoborus', 'aphanizomenon', 'paradileptus', 'keratella_quadrata', 'unknown_plankton', 'filament', 'dirt', 'keratella_cochlearis', 'asplanchna', 'eudiaptomus', 'hydra', 'asterionella', 'diaphanosoma']
class: kellikottia (519)
class: copepod_skins (33)
class: uroglena (953)
class: conochilus (264)
class: bosmina (81)
class: nauplius (1507)
class: unknown (245)
class: diatom_chain (17)
class: dinobryon (3321)
class: ceratium (814)
class: daphnia (721)
class: fragilaria (306)
class: maybe_cyano (1364)
class: leptodora (203)
class: daph

# Create sets (test and train)

In [9]:
sim.UpdateParams(testSplit=0.3)
sim.CreateTrainTestSets(ttkind=sim.params.ttkind)


# Create and Train Model
Custom parameter changes are made by acting directly on the params class through the `UpdateParams()` method.

In [11]:
sim.UpdateParams(totEpochs=10)
# print(sim.tt.trainXfeat.shape)
sim.Train()


Train on 112 samples, validate on 49 samples
Epoch 1/10
112/112 [==============================] - 0s 4ms/step - loss: 1.5551 - accuracy: 0.0714 - val_loss: 1.4246 - val_accuracy: 0.0408
Epoch 2/10
112/112 [==============================] - 0s 3ms/step - loss: 1.5214 - accuracy: 0.0714 - val_loss: 1.4213 - val_accuracy: 0.0408
Epoch 3/10
112/112 [==============================] - 0s 3ms/step - loss: 1.4897 - accuracy: 0.1161 - val_loss: 1.4182 - val_accuracy: 0.0408
Epoch 4/10
112/112 [==============================] - 0s 3ms/step - loss: 1.4606 - accuracy: 0.1607 - val_loss: 1.4151 - val_accuracy: 0.0408
Epoch 5/10
112/112 [==============================] - 0s 3ms/step - loss: 1.4329 - accuracy: 0.2054 - val_loss: 1.4122 - val_accuracy: 0.0408
Epoch 6/10
112/112 [==============================] - 0s 3ms/step - loss: 1.4077 - accuracy: 0.2768 - val_loss: 1.4093 - val_accuracy: 0.0408
Epoch 7/10
112/112 [==============================] - 0s 3ms/step - loss: 1.3845 - accuracy: 0.3393 - v

# Classification Report

In [12]:
sim.Report()

                  precision    recall  f1-score   support

         bosmina       0.00      0.00      0.00        27
       chaoborus       0.04      1.00      0.08         2
unknown_plankton       0.00      0.00      0.00        20

        accuracy                           0.04        49
       macro avg       0.01      0.33      0.03        49
    weighted avg       0.00      0.04      0.00        49



/opt/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


We now extract manually the training loss corresponding to the best weights, so that we can make sure that restarting the simulation works

In [13]:
def BestLoss(history):
    ''' Returns the training loss of the point where the validation loss was minimal'''
    return history['loss'][np.argmin(history['val_loss'])]
def InitLoss(history):
    ''' Returns the training loss of the point where the validation loss was minimal'''
    return history['loss'][0]

# initLoss = InitLoss(sim.history.history)
# bestLoss = BestLoss(sim.history.history)

print('Initial training loss:',InitLoss(sim.history.history))
print('Best    training loss:',BestLoss(sim.history.history))


Initial training loss: 1.5551090240478516
Best    training loss: 1.32373046875


# Start again from scratch
If we train again, the simulation does not start again where it ended, but it starts from scratch.
Since the default initialization is random, the initial value will be close to that of the previous run, but not the same.

In [14]:
sim.Train()

Train on 112 samples, validate on 49 samples
Epoch 1/10
112/112 [==============================] - 0s 4ms/step - loss: 1.0875 - accuracy: 0.4107 - val_loss: 1.1422 - val_accuracy: 0.4082
Epoch 2/10
112/112 [==============================] - 0s 3ms/step - loss: 1.0756 - accuracy: 0.4286 - val_loss: 1.1410 - val_accuracy: 0.4082
Epoch 3/10
112/112 [==============================] - 0s 3ms/step - loss: 1.0649 - accuracy: 0.4286 - val_loss: 1.1397 - val_accuracy: 0.4082
Epoch 4/10
112/112 [==============================] - 0s 3ms/step - loss: 1.0530 - accuracy: 0.4464 - val_loss: 1.1384 - val_accuracy: 0.4082
Epoch 5/10
112/112 [==============================] - 0s 3ms/step - loss: 1.0426 - accuracy: 0.4554 - val_loss: 1.1372 - val_accuracy: 0.4082
Epoch 6/10
112/112 [==============================] - 0s 3ms/step - loss: 1.0327 - accuracy: 0.4643 - val_loss: 1.1359 - val_accuracy: 0.4082
Epoch 7/10
112/112 [==============================] - 0s 3ms/step - loss: 1.0232 - accuracy: 0.4821 - v

In [15]:
print('Initial training loss:',InitLoss(sim.history.history))
print('Best    training loss:',BestLoss(sim.history.history))


Initial training loss: 1.087547745023455
Best    training loss: 0.9976513385772705


# Start from a previous state
We have two ways of not starting from scratch. We can either *(a)* define the full model and load the weight configuration, or *(b)* load the full model. We could also load the full model and then load a different weight configuration *(b+a)*.

I also show how to play around with some input parameters.



## (a) Define model and load weights from disk
We don't update the parameters, because we keep the same ones as in the previous runs, for comparison. The only thing we need to do, before calling `sim.Train()`, is to specify the file with the weight configuration through the `load_weights` parameter.

We will load the weights that minimized the test loss in the previous run.
You will see now that the initial loss is lower value than that of the two previous runs.

In [16]:
# Load the weights
sim.params.load_weights=sim.params.outpath+'/bestweights.hdf5'
sim.Train()

Loading weights from  out_example/bestweights.hdf5
Train on 112 samples, validate on 49 samples
Epoch 1/10
112/112 [==============================] - 1s 4ms/step - loss: 0.9900 - accuracy: 0.5000 - val_loss: 1.1297 - val_accuracy: 0.4082
Epoch 2/10
112/112 [==============================] - 0s 3ms/step - loss: 0.9827 - accuracy: 0.5179 - val_loss: 1.1285 - val_accuracy: 0.4082
Epoch 3/10
112/112 [==============================] - 0s 3ms/step - loss: 0.9763 - accuracy: 0.5089 - val_loss: 1.1272 - val_accuracy: 0.4082
Epoch 4/10
112/112 [==============================] - 0s 3ms/step - loss: 0.9700 - accuracy: 0.5179 - val_loss: 1.1260 - val_accuracy: 0.4082
Epoch 5/10
112/112 [==============================] - 0s 3ms/step - loss: 0.9631 - accuracy: 0.5179 - val_loss: 1.1248 - val_accuracy: 0.4082
Epoch 6/10
112/112 [==============================] - 0s 3ms/step - loss: 0.9577 - accuracy: 0.5357 - val_loss: 1.1236 - val_accuracy: 0.4082
Epoch 7/10
112/112 [==============================] 

In [17]:
print('Initial training loss:', InitLoss(sim.history.history))
print('Best    training loss:', BestLoss(sim.history.history))


Initial training loss: 0.990047982760838
Best    training loss: 0.9372863599232265


## (b) Loading full model from disk (CHECKPOINTING)
This is essentially what we do when checkpointing. At the end of each run, the entire model is saved (the default name is `'keras_model.h5'`). If we want to restart from there, we just need to load that model.

Since in this case we are checkpointing, we also show how to handle the simulation times.
Since when checkpointing one usually is starting a simulation from scratch, we will **define a new Ctrain class that loads all the parameters**.


In [18]:
# Load model, classes and parameters from the end of the previous run
model_from_previous  = sim.params.outpath+'/'+sim.params.saveModelName
params_from_previous = np.load(sim.params.outpath+'/params.npy', allow_pickle=True).item()
classes_from_previous = np.load(sim.params.outpath+'/classes.npy')

In [19]:
# Create a new class, just as if we were starting a new simulation that loaded a previous checkpoint
sim2=t.Ctrain()
sim2.params = params_from_previous
sim2.CreateOutDir()
sim2.LoadData(L=sim.params.L, class_select=sim.params.class_select, datakind=sim.params.datakind) # Should make a deep copy from sim
sim2.CreateTrainTestSets(ttkind=sim.params.ttkind)

datapaths: ['./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/', './data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.07.06/']
classes from datapaths: ['leptodora', 'aphanizomenon', 'nauplius', 'fish', 'ceratium', 'bosmina', 'keratella_quadrata', 'rotifers', 'hydra', 'polyarthra', 'daphnia_skins', 'uroglena', 'trichocerca', 'fragilaria', 'diaphanosoma', 'diatom_chain', 'daphnia', 'cyclops', 'asterionella', 'copepod_skins', 'unknown_plankton', 'dirt', 'unknown', 'filament', 'paradileptus', 'kellikottia', 'chaoborus', 'keratella_cochlearis', 'synchaeta', 'eudiaptomus', 'asplanchna', 'dinobryon', 'maybe_cyano', 'conochilus']
class: chaoborus (10)
class: bosmina (80)
class: unknown_plankton (71)


**Load model without training it, and make a classification report**. You can see that the model is loaded correctly, since it gives better than random predictions.

In [20]:
sim2.Train(train=False) #This only loads the model
sim2.Report()

Training took 0.002076117197672526 minutes
Saving the last model. These are not the best weights, they are the last ones. For the best weights use the callback output (bestweights.hdf5)]
                  precision    recall  f1-score   support

         bosmina       0.56      0.37      0.44        27
       chaoborus       0.06      1.00      0.12         2
unknown_plankton       0.00      0.00      0.00        20

        accuracy                           0.24        49
       macro avg       0.21      0.46      0.19        49
    weighted avg       0.31      0.24      0.25        49



/opt/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Now let's prolongate the run for some more steps.
Since we are prolongating a run, we also want to make sure that the timesteps are consistent

In [21]:
# Set initial epoch to the end of the previous run, and extend the total number of epochs (otherwise it won't run)
n_epochs_new = 20

sim2.params.initial_epoch=sim.history.epoch[-1]+1 if len(sim.history.epoch)>0 else 0
sim2.params.totEpochs=sim2.params.initial_epoch + n_epochs_new

# We do not want the Train() method to load weights from somewhere, 
# so we make sure that load_weights is set to None
# If we wanted to load the weights from somewhere, 
# it would be enough to give it the right file name (usually, 'bestweights.hdf5')
sim2.UpdateParams(load_weights = None)

In [22]:
print('Epochs before running:', sim.history.epoch)
sim2.Train()
print('Epochs after running:', sim2.history.epoch)


Epochs before running: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Train on 112 samples, validate on 49 samples
Epoch 11/30
112/112 [==============================] - 1s 6ms/step - loss: 1.2489 - accuracy: 0.4554 - val_loss: 1.4060 - val_accuracy: 0.4082
Epoch 12/30
112/112 [==============================] - 0s 3ms/step - loss: 1.2274 - accuracy: 0.4554 - val_loss: 1.4034 - val_accuracy: 0.4082
Epoch 13/30
112/112 [==============================] - 0s 3ms/step - loss: 1.2082 - accuracy: 0.4554 - val_loss: 1.4009 - val_accuracy: 0.4082
Epoch 14/30
112/112 [==============================] - 0s 3ms/step - loss: 1.1901 - accuracy: 0.4554 - val_loss: 1.3984 - val_accuracy: 0.4082
Epoch 15/30
112/112 [==============================] - 0s 3ms/step - loss: 1.1724 - accuracy: 0.4554 - val_loss: 1.3961 - val_accuracy: 0.4082
Epoch 16/30
112/112 [==============================] - 0s 3ms/step - loss: 1.1577 - accuracy: 0.4554 - val_loss: 1.3936 - val_accuracy: 0.4082
Epoch 17/30
112/112 [======================

# Use second model to predict on new, unseen data
Choose a class from the ones the classifier can recognize, and see whether the model is able to recognize it

In [23]:

# We only do this for 'image' models, because the format of the validation directory is wrong and I don't want to cope with that now
if ttkind == 'image':
    target = sim2.params.class_select[1]
    print('target:',target)

    testdir = 'data/1_zooplankton_0p5x/validation/tommy_validation/images/'+target
    im_names=np.array(glob.glob(testdir+'/*.jpeg'),dtype=object)
    npimages=hd.LoadImageList(im_names, L=sim2.params.L, show=False)

    probs=sim2.model.predict(npimages)
    predictions=probs.argmax(axis=1)  # The class that the classifier would bet on

    print('Predictions:',sim2.tt.lb.classes_[predictions])